In [2]:
import json
import os
from xmltodict import unparse

In [3]:
# change to where COCO is
ROOT = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\ObjectDetection-OneStageDet\\yolo\\COCO"
# where to save the .pkl files
DST = "./VOCdata"
# may change to 1, for there (seems to) be a slightly difference
# on bbox indexing between VOC and MSCOCO
BBOX_OFFSET = 0
# testing = 1 if testing
testing = 0

# only tested on val2017, the smallest file of MSCOCO
years = [
    "2017",
]

datasets = [
    "train",
]

labels =  ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
change_list = {'motorcycle': 'motorbike', 'airplane': 'aeroplane', 'tv': 'tvmonitor', 'couch': 'sofa', 'dining table': 'diningtable', 'potted plant': 'pottedplant'}


In [4]:
dataset = "train"
year = "2017"
if True:
    data = json.load(open(f"{ROOT}/annotations/instances_{dataset}{year}.json"))
    cate = {x['id']: x['name'] for x in data['categories']}

In [9]:
str = "["
for label in cate:
    str += f"\"{cate[label]}\", "
str += ']'
str

'["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush", ]'

In [22]:
    images = {}
    for image in data["images"]:
        images[image["id"]] = {
            # add more attributes if needed
            "file_name" : image["file_name"],
            "image_size" : {
                "height": image["height"],
                "width": image["width"],
            },
            "img_id" : image["id"],
            "obj" : []
        }

    for anno in data["annotations"]:
        # attach annotations to corresponding images
        images[anno["image_id"]]["obj"].append({
            "class_label": cate[anno["category_id"]],
            "bbox": anno["bbox"]
        })

In [23]:
    del_keys = []
    for image in images:
        if images[image]["obj"] == []:
            del_keys.append(image)
    for key in del_keys:
        print(key)
        del images[key]

262284
108169
176649
570045
191501
125997
69911
375363
325690
75083
26767
336777
217005
318596
34089
419106
281582
354041
279103
459408
244636
229782
434129
274233
136173
211423
266091
176149
334642
405104
340781
564317
403104
125084
70125
449546
197774
404462
544597
208708
176193
432647
289943
547760
540388
58133
545235
486632
503200
401623
237860
285068
109942
16903
466935
413120
50637
309571
68715
358795
220739
381842
104880
210766
244160
431026
104829
262189
576017
405945
383450
403279
487702
101623
572546
166524
222330
253520
113185
441863
129988
92604
84018
173685
382115
150616
450098
283147
91372
513149
345711
222757
338067
193704
24499
359184
521098
33554
331600
406217
79331
393212
68838
317130
362986
221828
325357
386613
293671
336873
16689
328084
568863
125182
500079
576354
361831
333198
534918
566025
507686
389811
444302
301765
252101
13789
516542
274957
345063
98268
29056
90280
539390
229981
39068
79913
120235
359465
10263
96923
441788
421673
379190
224970
21382
398454
2731

In [29]:
    val_annos = {}
    import sys
    sys.path.insert(0, "../yolo")
    import brambox.boxes as bbb
    for image in images:
        val_annos[f'{ROOT}/{dataset}{year}/{images[image]["file_name"]}'] = []
        for obj in images[image]["obj"]:
            
            tmp_obj = bbb.annotations.PascalVocAnnotation()
            tmp_obj.class_label = obj["class_label"]
            tmp_obj.x_top_left = int(max(obj["bbox"][0], 0))
            tmp_obj.y_top_left = int(max(obj["bbox"][1], 0))
            # maybe out of boundry! need to check?
            tmp_obj.width = int(obj["bbox"][2])
            tmp_obj.height = int(obj["bbox"][3])
            val_annos[f'{ROOT}/{dataset}{year}/{images[image]["file_name"]}'].append(tmp_obj)


In [32]:
    bbb.generate('anno_pickle', val_annos, f'../yolo/{DST}/onedet_cache/MSCOCO{dataset}{year}.pkl')

In [19]:
    del_keys = []
    for image in images:
        if images[image]['annotation']['object'] == []:
            #print(image)
            del_keys.append(image)
    for key in del_keys:
        del images[key]

In [34]:
    val_annos_fix_label = {}
    for image in val_annos:
        val_annos_fix_label[image] = []
        for anno in val_annos[image]:
            # map some coco label to voc labels
            if anno.class_label in change_list.keys():
                anno.class_label = change_list[anno.class_label]
            if anno.class_label in labels:
                val_annos_fix_label[image].append(anno)

In [35]:
    del_keys = []
    for image in val_annos_fix_label:
        if val_annos_fix_label[image] == []:
            del_keys.append(image)
    for key in del_keys:
        del val_annos_fix_label[key]

In [36]:
    len(val_annos_fix_label)

95279

In [42]:
    cnt = 0
    for image in val_annos_fix_label:
        for anno in val_annos_fix_label[image]:
            if anno.class_label == 'bird':
                print(anno)
                cnt += 1
        if cnt >= 10:
            break

Annotation {'bird' 0, [0, 525, 105, 89]}
Annotation {'bird' 0, [114, 348, 7, 5]}
Annotation {'bird' 0, [140, 350, 9, 7]}
Annotation {'bird' 0, [100, 404, 8, 11]}
Annotation {'bird' 0, [175, 344, 7, 6]}
Annotation {'bird' 0, [44, 268, 119, 111]}
Annotation {'bird' 0, [247, 269, 79, 50]}
Annotation {'bird' 0, [202, 237, 242, 111]}
Annotation {'bird' 0, [511, 409, 11, 11]}
Annotation {'bird' 0, [258, 391, 8, 4]}
Annotation {'bird' 0, [223, 388, 5, 3]}
Annotation {'bird' 0, [266, 396, 7, 2]}


In [44]:
    bbb.generate('anno_pickle', val_annos_fix_label, f'../yolo/{DST}/onedet_cache/MSCOCO{dataset}{year}_fix_label.pkl')

In [46]:
    import numpy as np
    random_seed = 123
    # random number generater
    rng = np.random.RandomState(random_seed)
    anno_format = "anno_pickle"
    class_label_map = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    identify = lambda f: f
    anno_filename = f'../yolo/{DST}/onedet_cache/MSCOCO{dataset}{year}_fix_label.pkl'
    kwargs = {}
    annos = bbb.parse(anno_format, anno_filename, identify=lambda f: f, class_label_map=class_label_map, **kwargs)
    keys = list(annos)

In [55]:
    cnt = 0
    for image in annos:
        for anno in annos[image]:
            if anno.class_label == "sofa":
                print(anno)
                cnt += 1
        if cnt >= 10:
            break

Annotation {'sofa' 0, [0, 0, 406, 375]}
Annotation {'sofa' 0, [2, 36, 496, 292]}
Annotation {'sofa' 0, [0, 71, 640, 348]}
Annotation {'sofa' 0, [0, 100, 639, 317]}
Annotation {'sofa' 0, [1, 240, 255, 151]}
Annotation {'sofa' 0, [94, 126, 345, 186]}
Annotation {'sofa' 0, [472, 135, 167, 179]}
Annotation {'sofa' 0, [62, 200, 210, 114]}
Annotation {'sofa' 0, [350, 213, 106, 120]}
Annotation {'sofa' 0, [16, 68, 481, 247]}


In [52]:
    k_shot_classes =['bird', 'bus', 'cow', 'motorbike', 'sofa'] # ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa"] #
    from collections import OrderedDict
    class2id = {k_shot_classes[i]: i for i in range(len(k_shot_classes))}
    class2file2numdet = {cls: OrderedDict() for cls in k_shot_classes}
    base_anno = {}
    for file, list_annos in annos.items():
        base_list = []
        for anno in list_annos:
            if anno.class_label in class2id:
                try:
                    # class to file to number of detection
                    class2file2numdet[anno.class_label][file] += 1
                except KeyError:
                    class2file2numdet[anno.class_label][file] = 1
            else:
                base_list.append(anno)
        if len(base_list) > 0:
            base_anno[file] = base_list

In [62]:
    k_shot = 3
    existing_sample_nums = np.zeros(len(k_shot_classes))

    current_class_id = 0
    k_shot_samples = set()
    finished_sampling = False

In [63]:
    while existing_sample_nums.min() < k_shot:
        assert existing_sample_nums[current_class_id] <= k_shot
        if existing_sample_nums[current_class_id] == k_shot:
            current_class_id = min(current_class_id+1, len(k_shot_classes))
            #print(existing_sample_nums)
            continue
        class_label = k_shot_classes[current_class_id]
        file2numdet = class2file2numdet[class_label]
        sample_file_id = rng.randint(len(file2numdet))
        sample_file_name = list(file2numdet.items())[sample_file_id][0]
        if sample_file_name in k_shot_samples:
            continue
        pred_sample_nums = existing_sample_nums.copy()
        for anno in annos[sample_file_name]:
            if anno.class_label in k_shot_classes:
                pred_sample_nums[class2id[anno.class_label]] += 1
        if pred_sample_nums.max() <= k_shot:
            print(sample_file_name, pred_sample_nums)
            k_shot_samples.add(sample_file_name)
            existing_sample_nums = pred_sample_nums
            print('class_label is {}'.format(class_label))

C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000075719.jpg [2. 0. 0. 0. 0.]
class_label is bird
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000059682.jpg [3. 0. 0. 0. 0.]
class_label is bird
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000566576.jpg [3. 2. 0. 0. 0.]
class_label is bus
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000421735.jpg [3. 3. 0. 0. 0.]
class_label is bus
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000043354.jpg [3. 3. 2. 0. 0.]
class_label is cow
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000084956.jpg [3. 3. 3. 0. 0.]
class_label is cow
C:\Users\AndrewHuang\Documents\GitHub\ObjectDetection-OneStageDet\yolo\COCO/train2017/000000262594.jpg [3. 3. 3. 1. 0.]
class_label is motorbike
C:\Users\AndrewHuan